### James Cooper | CSC 529 | Assignment 2

### Problem 1

#### One of the shortcomings of the results in Problem 2 from Assignment#1 is due to the fact of considering a single sample for training set as well as for test set (a single trial of 66% for training and 34% for testing was used to build the model). The objective of this exercise is to repeat the same experiment, but now with different (same size) samples as training and test sets (in other words, repeating the holdout procedure).

##### a. Repeat Problem 2.a&b from Assignment#1 on the Wine Recognition Dataset at least 30 times and report the means, variances, and    Confidence Intervals (CI) for the accuracy results on the training and testing sets. 


In [2]:
cd "C:/Users/James Cooper/Desktop/DePaul/Advanced Data Mining/Homework2"

C:\Users\James Cooper\Desktop\DePaul\Advanced Data Mining\Homework2


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
import seaborn as sns
from sklearn import preprocessing

Populating the interactive namespace from numpy and matplotlib


In [8]:
dat = pd.read_csv("Wine_data.csv", sep=',')

In [9]:
dat.head(5)


,Classes,Alcohol,Malic_Acid,Ash,Alcalinity_ash,Magnesium,Total_phenols,Flavanoids,Nonfavanoid_phenols,Proanthocyanins,Color Intensity,Hue,OD280_diluted_wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [10]:
data_2 = dat.drop('Classes', 1).values
classes_2 = dat['Classes'].astype('int').values

### Decision Tree Classifier

Below is just some exploratory analysis for getting an idea of how the data is classified with one decision tree

In [11]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_2, classes_2, test_size=0.34, random_state=33)

In [12]:
def perf_meas(X, y, classifier, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True):
    
    y_pred = classifier.predict(X)   
    if show_accuracy:
        print "Accuracy:{0:.3f}".format(metrics.accuracy_score(y, y_pred)),"\n"
    
    if show_classification_report:
        print "Classification report"
        print metrics.classification_report(y, y_pred),"\n"
      
    if show_confusion_matrix:
        print "Confusion matrix"
        print metrics.confusion_matrix(y, y_pred),"\n"

In [13]:
from sklearn import tree
dt = tree.DecisionTreeClassifier(criterion='entropy')
dt = dt.fit(X_train, y_train)

In [14]:
from sklearn import metrics
perf_meas(X_train, y_train, dt, show_confusion_matrix=True, show_classification_report=True)

Accuracy:1.000 

Classification report
             precision    recall  f1-score   support

          1       1.00      1.00      1.00        41
          2       1.00      1.00      1.00        48
          3       1.00      1.00      1.00        28

avg / total       1.00      1.00      1.00       117


Confusion matrix
[[41  0  0]
 [ 0 48  0]
 [ 0  0 28]] 



In [15]:
from sklearn import metrics
perf_meas(X_test, y_test, dt, show_confusion_matrix=True, show_classification_report=True)

Accuracy:0.902 

Classification report
             precision    recall  f1-score   support

          1       0.75      1.00      0.86        18
          2       1.00      1.00      1.00        23
          3       1.00      0.70      0.82        20

avg / total       0.93      0.90      0.90        61


Confusion matrix
[[18  0  0]
 [ 0 23  0]
 [ 6  0 14]] 



Below the Decision Tree Classifier is run 30 times to take a look at the scores for each random sample, the sample is taken without replacement. The classifier is run on the training and testing set looping through 30 times. The scores are stored in a list and the mean, variance, and confidence intervals are calculated. We can see that the Decision Tree overfits on the training data with perfect scores, while the test data is highly accurate, but has some variance.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import tree
from sklearn.metrics import accuracy_score

In [70]:
score_list_train = []
for i in range(30):
    from sklearn.cross_validation import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(data_2, classes_2, test_size=0.34, random_state = i)
    dtree = DecisionTreeClassifier()
    dtree_fit = dtree.fit(X_train, y_train)
    y_val_train = y_train
    y_hat_train = dtree.predict(X_train)
    scores_train = accuracy_score(y_val_train, y_hat_train)
    score_list_train.append(scores_train)
    print(scores_train)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


#### Mean, Variance, and Confidence Intervals of the Decision Tree Training Set:

In [71]:
print "The mean value of accuracy scores on the training set: {0:.6f}".format(np.mean(score_list_train))
print "The variance on the accuracy of the training set: {0:.6f}".format(np.var(score_list_train))

The mean value of accuracy scores on the training set: 1.000000
The variance on the accuracy of the training set: 0.000000


In [76]:
sorted_results_tr = np.array(score_list_train)
sorted_results_tr.sort()
lower_confidence = sorted_results_tr[int(0.025 * len(sorted_results_tr))]
upper_confidence = sorted_results_tr[int(0.975 * len(sorted_results_tr))]
print "95% confidence interval for training set: [{:0.4f} , {:0.4f}]".format(lower_confidence, upper_confidence)

95% confidence interval for training set: [1.0000 , 1.0000]


In [99]:
score_list_test = []
for i in range(30):
    from sklearn.cross_validation import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(data_2, classes_2, test_size=0.34, random_state = i)
    dtree = DecisionTreeClassifier()
    dtree_fit = dtree.fit(X_train, y_train)
    y_val = y_test
    y_hat = dtree.predict(X_test)
    scores = accuracy_score(y_val, y_hat)
    score_list_test.append(scores)
    print(scores)

0.950819672131
0.950819672131
0.852459016393
0.918032786885
0.885245901639
0.918032786885
0.934426229508
0.885245901639
0.885245901639
0.934426229508
0.901639344262
0.934426229508
0.950819672131
0.901639344262
0.983606557377
0.934426229508
0.83606557377
0.950819672131
0.918032786885
0.918032786885
0.918032786885
0.901639344262
0.819672131148
0.950819672131
0.934426229508
0.934426229508
0.868852459016
0.950819672131
0.950819672131
0.819672131148


#### The Mean, Variance, and Confidence Intervals of the Decision Tree test set:

In [74]:
print "The mean value of accuracy scores on the test set: {0:.6f}".format(np.mean(score_list_test))
print "The variance on the accuracy of the test set: {0:.6f}".format(np.var(score_list_test))

The mean value of accuracy scores on the test set: 0.916975
The variance on the accuracy of the test set: 0.001611


In [77]:
sorted_results_te = np.array(score_list_test)
sorted_results_te.sort()

lower_confidence = sorted_results_te[int(0.025 * len(sorted_results_te))]
upper_confidence = sorted_results_te[int(0.975 * len(sorted_results_te))]
print "95% confidence interval for training set: [{:0.4f} , {:0.4f}]".format(lower_confidence, upper_confidence)

95% confidence interval for training set: [0.8197 , 0.9836]


## Naive Bayes Classifier

In [81]:
from sklearn import naive_bayes
bclassifier = naive_bayes.GaussianNB()

In [96]:
scores_bayes_train = []
for i in range(30):
    from sklearn.cross_validation import train_test_split
    X_trainb, X_testb, y_trainb, y_testb = train_test_split(data_2, classes_2, test_size=0.34, random_state = i)
    bclassifier = naive_bayes.GaussianNB()
    bfitted = bclassifier.fit(X_trainb, y_trainb)
    y_valbtr = y_trainb
    y_predbtr = bfitted.predict(X_trainb)
    scoresbt = accuracy_score(y_valbtr, y_predbtr)
    scores_bayes_train.append(scoresbt)
print(scores_bayes_train)

[0.98290598290598286, 0.99145299145299148, 1.0, 1.0, 1.0, 1.0, 0.98290598290598286, 0.98290598290598286, 0.99145299145299148, 0.99145299145299148, 0.99145299145299148, 0.99145299145299148, 0.97435897435897434, 0.96581196581196582, 0.97435897435897434, 0.99145299145299148, 0.97435897435897434, 0.98290598290598286, 0.98290598290598286, 0.98290598290598286, 0.97435897435897434, 0.97435897435897434, 0.98290598290598286, 0.98290598290598286, 0.99145299145299148, 0.98290598290598286, 0.99145299145299148, 0.97435897435897434, 0.97435897435897434, 0.98290598290598286]


#### The Mean, Variance, and Confidence Intervals of the training set:

In [92]:
print "The mean value of accuracy scores on the training set: {0:.6f}".format(np.mean(scores_bayes_train))
print "The variance on the accuracy of the training set: {0:.6f}".format(np.var(scores_bayes_train))

The mean value of accuracy scores on the training set: 0.984900
The variance on the accuracy of the training set: 0.000081


In [97]:
sorted_resultsbtr = np.array(scores_bayes_train)
sorted_resultsbtr.sort()

lower_confidencebtr = sorted_resultsbtr[int(0.025 * len(sorted_resultsbtr))]
upper_confidencebtr = sorted_resultsbtr[int(0.975 * len(sorted_resultsbtr))]
print "95% confidence interval for training set: [{:0.4f}, {:0.4f}]".format(lower_confidencebtr, upper_confidencebtr)

95% confidence interval for training set: [0.9658, 1.0000]


In [95]:
scores_bayes_test = []
for i in range(30):
    from sklearn.cross_validation import train_test_split
    X_trainb, X_testb, y_trainb, y_testb = train_test_split(data_2, classes_2, test_size=0.34, random_state = i)
    bclassifier = naive_bayes.GaussianNB()
    bfitted = bclassifier.fit(X_trainb, y_trainb)
    y_valb = y_testb
    y_predb = bfitted.predict(X_testb)
    scoresb = accuracy_score(y_valb, y_predb)
    scores_bayes_test.append(scoresb)
print(scores_bayes_test)

[0.95081967213114749, 0.98360655737704916, 0.96721311475409832, 0.95081967213114749, 0.96721311475409832, 0.91803278688524592, 1.0, 1.0, 0.96721311475409832, 0.95081967213114749, 0.90163934426229508, 0.95081967213114749, 0.95081967213114749, 0.98360655737704916, 1.0, 0.98360655737704916, 0.98360655737704916, 1.0, 0.96721311475409832, 0.96721311475409832, 0.98360655737704916, 0.96721311475409832, 0.96721311475409832, 1.0, 0.96721311475409832, 0.98360655737704916, 0.98360655737704916, 1.0, 0.96721311475409832, 0.96721311475409832]


#### The Mean, Variance, and Confidence Intervals of the test set:

In [87]:
print "The mean value of accuracy scores on the test set: {0:.6f}".format(np.mean(scores_bayes_test))
print "The variance on the accuracy of the test set: {0:.6f}".format(np.var(scores_bayes_test))

The mean value of accuracy scores on the test set: 0.971038
The variance on the accuracy of the test set: 0.000532


In [88]:
sorted_resultsbte = np.array(scores_bayes_test)
sorted_resultsbte.sort()

lower_confidencebte = sorted_resultsbte[int(0.025 * len(sorted_resultsbte))]
upper_confidencebte = sorted_resultsbte[int(0.975 * len(sorted_resultsbte))]
print "95% confidence interval for training set: [{:0.4f} , {:0.4f}]".format(lower_confidencebte, upper_confidencebte)

95% confidence interval for training set: [0.9016 , 1.0000]


#### b.	Using a pair t-test, compare the mean accuracy of the Naïve Bayes and the mean accuracy of the Decision tree and discuss the results.

In [111]:
len(X_train)

117

In [112]:
len(X_trainb)

117

In [121]:
from scipy.stats import ttest_ind, ttest_ind_from_stats
dtr_mean = np.mean(score_list_train)
dtr_std = np.std(score_list_train)
btr_mean = np.mean(scores_bayes_train)
btr_std = np.std(scores_bayes_train)
ttest_ind_from_stats(mean1 = dtr_mean, std1 = dtr_std, nobs1 = 30, mean2 = btr_mean, std2 = btr_std, nobs2 = 30)



Ttest_indResult(statistic=9.1752827850597445, pvalue=6.8216106280406834e-13)

In [120]:
ttest_ind(score_list_train, scores_bayes_train)

Ttest_indResult(statistic=9.1727755346276183, pvalue=5.8937757110133497e-13)

Using the function ttest_ind_from_stats, the null hypothesis is that two independent samples have identical average expected values. The function compares the mean and standard deviations of the scores from the decision trees and bayes models that were used with the wine data. We can see that we get a p value low enough to reject the null hypothesis, showing that the models do not have the same average expected values.